In [2]:
import numpy as np
from pypianoroll import Multitrack, Track
from matplotlib import pyplot as plt
#import tensorflow as tf

In [3]:
import os
print (os.getcwd())
print(os.getcwd())

D:\1projects\MusicGan\styleMuseGAN
D:\1projects\MusicGan\styleMuseGAN


In [4]:
all_track=[]

In [5]:
i=0
k=0
c=20000
batch_size=500

In [6]:
# analizing notes
#input a Multitrack object 
#ex:
#another_multitrack = Multitrack('./data2/rock-39.mid')
#note_distribution(another_multitrack)
#return note_distribution of the whole song which order is C #C D #D ... bB B

def note_distribution(another_multitrack):
    #notes_per_beat is the minus 24 mod14 product C(Do) to be zero is better for the further task
    notes_per_beat=[]
    length=len(another_multitrack.get_merged_pianoroll())
    merged=another_multitrack.get_merged_pianoroll()
    #print(merged)

    for i in range(length):
        #if (length-i)%1000==0:
            #print(length-i)
        #print('ori',np.nonzero(another_multitrack.get_merged_pianoroll()[i])[0])
        temp=list(np.nonzero(merged[i])[0])              
        for j in range(len(temp)):
            temp[j]=(temp[j]-24)%12
        notes_per_beat.append(temp)
        #print(temp)
        #print('new',temp)
        #print(list(np.nonzero(another_multitrack.get_merged_pianoroll()[i])[0])+1)
    #np.nonzero(another_multitrack.get_merged_pianoroll())
    length=len(another_multitrack.get_merged_pianoroll())
    note_distributions=[]
    for i in range(12):
        note_distributions.append(0)
    #print(note_distributions)
    for i in range(length):
        #print(notes_per_beat[i])
        for j in range(12):
            #print(j,notes_per_beat[i].count(j))
            note_distributions[j]= notes_per_beat[i].count(j)+note_distributions[j]
    #print(note_distributions)
    sum=0
    for ele in note_distributions:
        sum=sum+ele
    note_distribution_frequency=[]
    for ele in note_distributions:
        note_distribution_frequency.append(ele/sum)
    
    return note_distributions,note_distribution_frequency


In [7]:

class LPDDataset:
    def __init__(self):
        with open('file_path2.txt','rb') as f:
            self.files=f.readlines()
            
    def __getitem__(self, idx):
        file = self.files[idx]
        loaded = Multitrack(file[:-2].decode('utf-8'))
        return loaded
    
    def __len__(self):
        return len(self.files)


def load(x):
    return x
"""
#平行畫運算

from concurrent.futures import ProcessPoolExecutor

with ProcessPoolExecutor() as executer:
    for idx, output in enumerate(executer.map(load, data)):
        print(output)
        break


"""


'\n#平行畫運算\n\nfrom concurrent.futures import ProcessPoolExecutor\n\nwith ProcessPoolExecutor() as executer:\n    for idx, output in enumerate(executer.map(load, data)):\n        print(output)\n        break\n\n\n'

In [8]:
data = LPDDataset()
#single track np_array的call 法 第一個dimension為長度axis 第二個為音 axis
roll=data[1].tracks[2].pianoroll
#roll=np.transpose(roll)
print(data[1])
print(data[1].tracks[0].pianoroll[129][12])
print(len(data))

tracks : [Drums, Piano, Guitar, Bass, Strings],
tempo : [105.00157502 105.00157502 105.00157502 ... 105.00157502 105.00157502
 105.00157502],
downbeat : [ True False False ... False False False],
beat_resolution : 24,
name : 05f21994c71a5f881e64f45c8d706165
0
21425


In [12]:
#find longest length
length=0
ten_thousand_down=0
two_thousand_down=0
two_thousand_up=0
for i in range(len(data)):
    temp=data[i]
    if temp.tracks[0].pianoroll.shape[0]<10000 and  temp.tracks[3].pianoroll.shape[0]<10000:
        ten_thousand_down=ten_thousand_down+1
    elif temp.tracks[0].pianoroll.shape[0]<20000 and temp.tracks[3].pianoroll.shape[0]<20000:
        two_thousand_down=two_thousand_down+1
    else:
        two_thousand_up=three_thousand_up+1
        """
      if length<data[i].tracks[0].pianoroll.shape[0]:
        length=data[i].tracks[0].pianoroll.shape[0]
        print(length)   
        """
   

In [17]:
print(ten_thousand_down)
print(three_thousand_down)
print(three_thousand_up)
print(length)
print('get '+ '15008 '+'songs after downsampling')

11673
9478
274
0
get 15008 songs after downsampling


In [84]:
sdf[0]=1
print(sdf[0])

1.0


In [87]:
temp=np.zeros((1,128))
i=23
for j in range(5):
    #print(data[i].tracks[j].pianoroll.shape)
    #print(np.vstack((data[23].tracks[j].pianoroll,temp)).shape)
    temproll=np.vstack((data[i].tracks[j].pianoroll,temp))
    #print(temproll.shape[0])
    temproll=np.vstack((temproll,np.zeros((20000-temproll.shape[0],128))))
    #print(temproll.shape)
print(data[i].tracks[2].pianoroll)   



###################
#Checking original data
################### 
#for sd in range(5000):
   # print(data[i].tracks[2].pianoroll[2000+sd])
    
temproll=np.vstack((data[i].tracks[2].pianoroll,temp))    
temproll=np.vstack((temproll,np.zeros((20000-temproll.shape[0],128))))
#roll is a np_array
#number is the number of 
#return a array_with shape 20000*128
def downsampling(number, roll):
    new=np.zeros((20000,128))
    for c in range(int(20000/number)):
        #計算某個因出現的次數
        sdf=np.zeros(128)
        for j in range(128):
            for i in range(number):
                if roll[c*number+i][j]!=0:
                    sdf[j]=sdf[j]+roll[c*number+i][j]
                    #print('hi')
            #print(sdf)
        #開始塞入new _roll which is a downsampling version of roll
        for j in range(128):
            for i in range(number):
                new[c*number+i][j]=sdf[j]
    return new
     
    
"""
A = array([[0, 1, 2], [0, 2, 0]])
X = array([[0, 1, 2], [1, 2, 0], [2, 1, 2], [3, 2, 0]])
A = vstack((A, X[X[:,0] < 3]))

# returns: 
array([[0, 1, 2],
       [0, 2, 0],
       [0, 1, 2],
       [1, 2, 0],
       [2, 1, 2]])
       """

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


'\nA = array([[0, 1, 2], [0, 2, 0]])\nX = array([[0, 1, 2], [1, 2, 0], [2, 1, 2], [3, 2, 0]])\nA = vstack((A, X[X[:,0] < 3]))\n\n# returns: \narray([[0, 1, 2],\n       [0, 2, 0],\n       [0, 1, 2],\n       [1, 2, 0],\n       [2, 1, 2]])\n       '

In [94]:
new=downsampling(24,temproll)       
###################
#Checking original data
################### 
for sd in range(48):
   print(data[i].tracks[2].pianoroll[2400+sd])

print('#####################')
#####
#check new
###
for sd in range(48):
    print(new[2400+sd])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0 71  0  0 77  0  0  0  0 70  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0 71  0  0 77  0  0  0  0 70  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0 71  0  0 77  0  0  0  0 70  0  0  0  0  0  0  0  0  0  0

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

   0.   0.]
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 568.   0.   0.
 616.   0.   0.   0.   0. 560.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.]
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 568.   0.   0.
 616.   0.   0.   0.   0. 560.   0.   0.   0.   0.   

In [45]:
A = np.array([[0, 1, 2], [0, 2, 0]])
X = np.array([[0, 1, 2], [1, 2, 0], [2, 1, 2], [3, 2, 0]])
A = np.vstack((A, X))
print(A)

[[0 1 2]
 [0 2 0]
 [0 1 2]
 [1 2 0]
 [2 1 2]
 [3 2 0]]


In [24]:
print(roll.shape)
for j in range (24):
    for i in range(128):
        print(roll[1025+j][i])

(7896, 128)
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [9]:

file=open('file_path2.txt','rb')
i=0
note_dis=[]
length=0
for line in file:
    i=i+1    
    #c=c-1
    #if c%100==0:
        #print(c/100)
    #print(c)
    #print(line[:-2].decode('utf-8'))
    loaded = Multitrack(line[:-2].decode('utf-8'))
    #print(loaded)
    #print('\n')
    #if i>5:
     #   break
    #print(loaded)
    #file_names.append(line[:-2].decode('utf-8'))
    #note_dis.append(note_distribution(loaded))
    #temp=loaded.get_merged_pianoroll()
    #####
    #Computing longest length
    ###
    #print(temp.shape)
    #if length<temp.shape[0]:
     #   length=temp.shape[0]
    if i%20==0:
        print(i)
print(length)



tracks : [Drums, Piano, Guitar, Bass, Strings],
tempo : [122.00180563 122.00180563 122.00180563 ... 122.00180563 122.00180563
 122.00180563],
downbeat : [ True False False ... False False False],
beat_resolution : 24,
name : b97c529ab9ef783a849b896816001748


tracks : [Drums, Piano, Guitar, Bass, Strings],
tempo : [105.00157502 105.00157502 105.00157502 ... 105.00157502 105.00157502
 105.00157502],
downbeat : [ True False False ... False False False],
beat_resolution : 24,
name : 05f21994c71a5f881e64f45c8d706165


tracks : [Drums, Piano, Guitar, Bass, Strings],
tempo : [67.00002903 67.00002903 67.00002903 ... 67.00002903 67.00002903
 67.00002903],
downbeat : [ True False False ... False False False],
beat_resolution : 24,
name : 04266ac849c1d3814dc03bbf61511b33


tracks : [Drums, Piano, Guitar, Bass, Strings],
tempo : [114.2857415  115.00002875 115.00002875 ... 115.00002875 115.00002875
 115.00002875],
downbeat : [ True False False ... False False False],
beat_resolution : 24,
name : f

In [14]:
print(len(note_dis))
print(note_dis[0])

21425
([7215, 8, 7598, 0, 3028, 7203, 1241, 8557, 0, 8972, 0, 3465], [0.15257893289910546, 0.0001691796899782181, 0.16067841055681265, 0.0, 0.06403451265675555, 0.15232516336413812, 0.026243999407871085, 0.18095882589295156, 0.0, 0.18973502231057163, 0.0, 0.07327595322181572])


In [2]:
print(loaded)

NameError: name 'loaded' is not defined

In [16]:
#save data
"""
import pickle
file = open('note_dis_dataset.pke', 'wb')
pickle.dump(note_dis, file)
file.close()
"""


In [4]:
#load data
import pickle

with open('note_dis_dataset.pke', 'rb') as file:
    note_dis =pickle.load(file)

In [26]:
dataset = tf.data.Dataset.from_tensor_slices((note_dis))
dataset=dataset.shuffle(50).batch(batch_size).repeat(10)
iterator =dataset.make_one_shot_iterator()
next_element=iterator.get_next()

In [36]:
with tf.Session() as sess:
    for i in range(10):
        print(sess.run(next_element)[1])

[[8.9310000e+03 4.9050000e+03 2.2153000e+04 9.6400000e+02 7.5690000e+03
  1.2387000e+04 8.4500000e+02 1.3954000e+04 1.1540000e+03 1.7811000e+04
  7.5520000e+03 7.0600000e+02]
 [9.0275042e-02 4.9580012e-02 2.2392374e-01 9.7441655e-03 7.6507866e-02
  1.2520848e-01 8.5413065e-03 1.4104781e-01 1.1664696e-02 1.8003456e-01
  7.6336034e-02 7.1362867e-03]]
[[6.06400000e+03 5.75100000e+03 6.40600000e+03 4.59200000e+03
  1.37750000e+04 3.30800000e+03 1.02750000e+04 5.28000000e+03
  7.78200000e+03 9.67500000e+03 5.21000000e+02 1.17930000e+04]
 [7.11553320e-02 6.74825758e-02 7.51683861e-02 5.38828000e-02
  1.61636665e-01 3.88162695e-02 1.20567456e-01 6.19558319e-02
  9.13144499e-02 1.13527022e-01 6.11344492e-03 1.38379768e-01]]
[[1.6230000e+03 0.0000000e+00 4.4300000e+02 1.1410000e+03 0.0000000e+00
  5.7800000e+02 0.0000000e+00 1.5670000e+03 1.3590000e+03 0.0000000e+00
  2.6100000e+02 1.6000000e+02]
 [2.2756590e-01 0.0000000e+00 6.2114414e-02 1.5998317e-01 0.0000000e+00
  8.1043184e-02 0.0000000e+